In [36]:
#Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli, pauli_basis, SuperOp, PTM, Operator
from qiskit.providers.aer.noise import NoiseModel, pauli_error
from qiskit.providers.fake_provider import FakeVigoV2
from qiskit.providers.aer import Aer, AerSimulator
from random import random

#python imports
from random import choice, choices
from itertools import product, permutations, cycle
from scipy.optimize import curve_fit, nnls
from matplotlib import pyplot as plt
import numpy as np

plt.style.use("ggplot")

In [38]:
from TrotterExample import get_noise_model, executor
import qiskit.providers.fake_provider as fake_provider # FakeMelbourneV2, FakeCasablancaV2, FakeVigoV2, FakeLagosV2, FakeGuadalupeV2, FakeGuadalupe, FakeGeneva

prog = QuantumCircuit(5)
prog.cnot(0,1)
prog.measure_all()
backend = fake_provider.FakeVigoV2()
#backend = Aer.get_backend('qasm_simulator')
(noise_model, twoqubit_error_template, singlequbit_error_template) = get_noise_model()
#noise_model = NoiseModel()
counts = executor([prog], backend, 1024, noise_model=noise_model)
print(counts)
print(noise_model)

{'00000': 975, '00011': 35, '00001': 4, '00010': 10}
NoiseModel:
  Basis gates: ['cx', 'cz', 'id', 'rz', 'sx']
  Instructions with noise: ['cx', 'cz']
  All-qubits errors: ['cx', 'cz']
